<a href="https://colab.research.google.com/github/ssuxmin/tmp/blob/main/automl_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/test.csv')

In [4]:
print(sorted(train['범죄발생지'].unique()))
print(sorted(test['범죄발생지'].unique()))

['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']
['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']


# 시드(seed) 고정  
매번 고정된 결과를 얻기 위해서 사용합니다.  
시드를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나올 수 있습니다.

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [36]:
x_train = train.drop(['ID', 'TARGET'], axis = 1)
y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

In [7]:
x_train = train.drop(['ID'], axis = 1)
# y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

In [37]:
x_train

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
0,9,화요일,10,137,8.0,2.611124,0.000000,0.0,0.00,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
1,11,화요일,6,438,13.0,3.209093,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
2,8,일요일,6,1729,47.0,1.619597,0.000000,0.0,0.00,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도
3,5,월요일,6,2337,53.0,1.921615,11.375000,0.0,0.00,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지
4,9,일요일,11,1439,41.0,1.789721,0.000000,0.0,0.00,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84401,4,일요일,7,336,11.0,3.808190,99.111111,0.0,0.00,165.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
84402,8,목요일,12,2149,38.0,1.458490,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,차도
84403,7,일요일,6,29,46.0,2.944913,105.888889,0.0,0.00,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
84404,1,화요일,11,536,25.0,0.493679,2.285714,8.6,10.75,330.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,주거지


In [38]:
x_test

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
0,9,금요일,5,927,28.0,1.570654,19.625000,0.0,0.0,165.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,차도
1,5,수요일,3,926,28.0,1.712457,21.444444,0.0,0.0,175.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,식당
2,5,월요일,6,1437,33.0,0.447496,25.200000,0.0,0.0,290.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
3,11,화요일,1,1739,31.0,0.878585,0.000000,0.0,0.0,285.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
4,10,목요일,10,830,15.0,0.496423,26.142857,0.0,0.0,95.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17284,11,금요일,9,1138,31.0,2.500387,0.000000,0.0,0.0,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
17285,3,월요일,1,148,9.0,2.234593,0.000000,0.0,0.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
17286,10,금요일,9,2347,53.0,0.360005,6.857143,0.0,0.0,340.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
17287,9,목요일,4,548,13.0,3.993724,0.000000,0.0,0.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지


In [39]:
ordinal_features = ['요일', '범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(x_train[feature])
    x_train[feature] = le.transform(x_train[feature])

    # x_train데이터에서 존재하지 않았던 값이 x_test 데이터에 존재할 수도 있습니다.
    # 따라서 x_test 데이터를 바로 변형시키지 않고 고윳값을 확인후 x_test 데이터를 변환합니다.
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_test[feature] = le.transform(x_test[feature])

In [11]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
from pycaret import classification
classification.setup(data=x_train,target='TARGET',
                 remove_outliers=True)

In [14]:
best = classification.compare_models(n_select=5,fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5492,0.6990,0.5492,0.5460,0.5389,0.2884,0.2942,0.8280
gbc,Gradient Boosting Classifier,0.5477,0.6970,0.5477,0.5462,0.5373,0.2863,0.2924,0.8300
xgboost,Extreme Gradient Boosting,0.5435,0.6925,0.5435,0.5392,0.5335,0.2801,0.2853,0.3540
ada,Ada Boost Classifier,0.5428,0.6861,0.5428,0.5406,0.5333,0.2809,0.2859,0.3880
rf,Random Forest Classifier,0.5238,0.6739,0.5238,0.5168,0.5133,0.2491,0.2534,5.0520
et,Extra Trees Classifier,0.4997,0.6523,0.4997,0.4909,0.4900,0.2134,0.2163,4.4180
nb,Naive Bayes,0.4687,0.6106,0.4687,0.4645,0.4632,0.1761,0.1774,0.2160
lda,Linear Discriminant Analysis,0.4560,0.6001,0.4560,0.4178,0.3868,0.0935,0.1142,0.2860
ridge,Ridge Classifier,0.4543,0.0000,0.4543,0.3973,0.3703,0.0826,0.1072,0.1840
lr,Logistic Regression,0.4417,0.5730,0.4417,0.4187,0.3719,0.0673,0.0846,1.4240


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [33]:
# 훈련 데이터와 테스트 데이터의 특성 수 확인
print(x_train.shape[1])  # 훈련 데이터의 특성 수
print(x_test.shape[1])  # 테스트 데이터의 특성 수

19
18


In [34]:
# 'TARGET' 열을 제외하고 테스트 데이터의 열 순서를 동일하게 맞춰주는 예시 코드:
x_test = x_test[x_train.drop('TARGET', axis=1).columns]
x_test

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
0,9,0,5,927,28.0,1.570654,19.625000,0.0,0.0,165.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,10
1,5,2,3,926,28.0,1.712457,21.444444,0.0,0.0,175.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3
2,5,3,6,1437,33.0,0.447496,25.200000,0.0,0.0,290.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7
3,11,6,1,1739,31.0,0.878585,0.000000,0.0,0.0,285.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
4,10,1,10,830,15.0,0.496423,26.142857,0.0,0.0,95.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17284,11,0,9,1138,31.0,2.500387,0.000000,0.0,0.0,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
17285,3,3,1,148,9.0,2.234593,0.000000,0.0,0.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
17286,10,0,9,2347,53.0,0.360005,6.857143,0.0,0.0,340.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
17287,9,1,4,548,13.0,3.993724,0.000000,0.0,0.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [35]:
x_train

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,9,6,10,137,8.0,2.611124,0.000000,0.0,0.00,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,2
1,11,6,6,438,13.0,3.209093,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0
2,8,4,6,1729,47.0,1.619597,0.000000,0.0,0.00,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,6,1
3,5,3,6,2337,53.0,1.921615,11.375000,0.0,0.00,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,7,1
4,9,4,11,1439,41.0,1.789721,0.000000,0.0,0.00,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84401,4,4,7,336,11.0,3.808190,99.111111,0.0,0.00,165.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,10,1
84402,8,1,12,2149,38.0,1.458490,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10,0
84403,7,4,6,29,46.0,2.944913,105.888889,0.0,0.00,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0
84404,1,6,11,536,25.0,0.493679,2.285714,8.6,10.75,330.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,7,1


In [22]:
from lightgbm import LGBMClassifier

In [23]:
# LightGBM 모델 인스턴스화
lgbm_model = LGBMClassifier()

# 모델 학습
lgbm_model.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [47]:
import lightgbm as lgb

# LightGBM 모델 초기화
lgbm_model = lgb.LGBMClassifier(n_estimators=100)


In [48]:
# 모델 훈련
lgbm_model.fit(x_train, y_train)

# 테스트 데이터에 대한 예측
pred = lgbm_model.predict(x_test)

In [41]:
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)


x_train shape: (84406, 18)
x_test shape: (17289, 18)


In [44]:
probas = []
for model in best:
    probas.append(model.predict_proba(x_test))

pred = [np.argmax(i) for i in np.array(probas).mean(axis=0)]

In [49]:
# 제출 파일을 읽어옵니다.
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/sample_submission.csv') 

In [50]:
# 예측한 값을 TARGET 컬럼에 할당합니다.
submit['TARGET'] = pred
submit.head()

,ID,TARGET
0,TEST_00000,2
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,0
4,TEST_00004,0


In [51]:
print(submit.head())


           ID  TARGET
0  TEST_00000       2
1  TEST_00001       0
2  TEST_00002       0
3  TEST_00003       0
4  TEST_00004       0


In [52]:
# 'ID' 열을 포함하여 제출 파일을 다시 읽어옵니다.
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/sample_submission.csv')

print(sub.columns)

Index(['ID', 'TARGET'], dtype='object')


In [53]:
# 예측한 결과를 파일로 저장합니다. index 인자의 값을 False로 설정하지 않으면 제출이 정상적으로 진행되지 않습니다.
sub.set_index('ID',inplace=True)
sub['TARGET'] = pred

In [54]:
# 'ID' 컬럼이 제대로 포함되어 있는지 확인합니다.
if 'ID' not in sub.columns:
    sub.insert(0, 'ID', sub.index)

In [55]:
print(sub.head())

                    ID  TARGET
ID                            
TEST_00000  TEST_00000       2
TEST_00001  TEST_00001       0
TEST_00002  TEST_00002       0
TEST_00003  TEST_00003       0
TEST_00004  TEST_00004       0


In [56]:
# 수정된 데이터프레임을 파일로 저장합니다.
sub.to_csv('./lightbgm_submit1.csv', index=False)


# 다시 구현(gbc)


In [58]:
from sklearn.ensemble import GradientBoostingClassifier

In [59]:
# Gradient Boosting Classifier 모델 초기화
gb_model = GradientBoostingClassifier(n_estimators=100)

In [60]:
# 모델 훈련
gb_model.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [61]:
# 테스트 데이터에 대한 예측
pred = gb_model.predict(x_test)

In [62]:
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

x_train shape: (84406, 18)
x_test shape: (17289, 18)


In [64]:
# 제출 파일을 읽어옵니다.
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/범죄유형/sample_submission.csv') 

In [65]:
# 예측한 값을 TARGET 컬럼에 할당합니다.
submit['TARGET'] = pred
submit.head()

,ID,TARGET
0,TEST_00000,2
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,0
4,TEST_00004,0


In [66]:
# 예측한 결과를 파일로 저장합니다. index 인자의 값을 False로 설정하지 않으면 제출이 정상적으로 진행되지 않습니다.
submit.to_csv('submit.csv', index = False)